# QLによるビン取りゲーム（対戦）

## ライブラリのインストール

In [1]:
import numpy as np

## シミュレータクラスの設定

In [2]:
class MySimulator():
    def __init__(self, num_bottle):
        self._nb = num_bottle
        self.reset()
#初期化
    def reset(self):
        self._state = 0
        return self._state
#行動による状態変化
    def step(self, action, turn):
        self._state = self._state + action + 1
        rewards = [0,0]
        done = False
        if (self._state>=self._nb):
            self._state = self._nb
            rewards[turn] = -1
            rewards[(turn+1)%2] = 1
            done = True
        return self._state, rewards, done

## Q値クラスの設定

In [3]:
class MyQTable():
    def __init__(self, num_bottle):
        self._nb = num_bottle
        QV0=np.zeros((3,self._nb+1), dtype=np.float32)
        QV1=np.zeros((3,self._nb+1), dtype=np.float32)
        self._QVs = [QV0, QV1]
#    self._Qtable = np.zeros((2, 2))
#行動の選択
    def get_action(self, state, epsilon, turn):
        qv= self._QVs[turn]
        if epsilon > np.random.uniform(0, 1):#徐々に最適行動のみをとる、ε-greedy法
            next_action = np.random.choice([0, 1, 2])
        else:
            a = np.where(qv[:,state]==qv[:,state].max())[0]
            next_action = np.random.choice(a)
        return next_action
    def update_Qtable(self, act, reward, pos, pos_old, turn):
        qv= self._QVs[turn]
        alpha = 0.5
        gamma = 0.9
        maxQ = np.max(qv[:,pos])
        qv[act,pos_old] = (1-alpha)*qv[act,pos_old]+alpha*(reward + gamma*maxQ);
        self._QVs[turn] = qv
    def show_Qtable(self):
        print('Agent0')
        print(self._QVs[0].T)
        print('Agent1')
        print(self._QVs[1].T)
        for j in range(2):
            print("Agent", j)
            qv = self._QVs[j]
            for i in range(self._nb):
                a = np.where(qv[:,i]==qv[:,i].max())[0]
#                a = np.where(self._QVs[j][i]==self._QVs[j][i].max())[0]
                print('残り本数',self._nb-i,'取る数',a+1,'必勝法',(self._nb-i-1)%4,'なんでもよい' if (self._nb-i-1)%4 == 0 else \
                      ('不定' if a.size >1 else ('正解' if (self._nb-i-1)%4 == a+1 else '不正解')))


## その他の設定

In [4]:
#num_episodes = 100  #総試行回数
num_bottle = 9
env = MySimulator(num_bottle)
tab = MyQTable(num_bottle)

tab._QVs[0] = np.loadtxt('Qvalue0.txt')
tab._QVs[1] = np.loadtxt('Qvalue1.txt')

## 対戦

In [12]:
state = env.reset()
print('=== ビン取りゲーム ===')
you = input('先攻（0） or 後攻（1）を選択：')
you = int(you)
me = 0 if you==1 else 1
turn = 0
actions = [0,0]

while(1):
    print(f'残り{num_bottle-env._state}本です．')
    if turn == you:
        pos = input('何本取りますか？ ("1- 3")：')
        print(f'{int(pos)-1}本取りました．')
        actions[0] = int(pos)-1
        state, rewards, done = env.step(actions[0], 0)
        if (done==True):
            print('あなたの負け')
            break
        turn = me
    else:
        actions[1] = tab.get_action(state, 0, turn=1)
        print(f'{actions[1]+1}本取りました．')
        state, rewards, done = env.step(actions[1], 1)
        if (done==True):
            print('あなたの勝ち')
            break
        turn = you



=== ビン取りゲーム ===
先攻（0） or 後攻（1）を選択：0
残り9本です．
何本取りますか？ ("1- 3")：1
0本取りました．
残り8本です．
3本取りました．
残り5本です．
何本取りますか？ ("1- 3")：2
1本取りました．
残り3本です．
2本取りました．
残り1本です．
何本取りますか？ ("1- 3")：1
0本取りました．
あなたの負け
